In [1]:
import matplotlib.pyplot as plt  # plots
import numpy as np  # vectors and matrices
import pandas as pd  # tables and data manipulations
import seaborn as sns  # more plots
sns.set()
import warnings  
warnings.filterwarnings('ignore')
from itertools import product  # some useful functions
import scipy.stats as scs
from dateutil.relativedelta import relativedelta  # working with dates with style
from scipy.optimize import minimize  # for function minimization
from sklearn.model_selection import TimeSeriesSplit  # you have everything done for you 
from sklearn.model_selection import train_test_split
import datetime
from sklearn import metrics

In [2]:
df_G2 = pd.read_csv('G2.csv.xls', sep = ',') 
df_G2['date'] = df_G2['date'].map(lambda x: datetime.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S"))
# print(df_G2)
print(df_G2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    156 non-null    datetime64[ns]
 1   SO2op1  156 non-null    float64       
 2   SO2op2  156 non-null    float64       
 3   T       156 non-null    float64       
 4   SO2     156 non-null    float64       
 5   NO2     156 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 7.4 KB
None


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


y = df_G2.dropna()['SO2']
X = df_G2.dropna().drop(['SO2'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.33)

time_train = X_train.dropna()['date']
X_train = X_train.drop(['date'], axis=1)
time_valid = X_valid.dropna()['date']
X_valid = X_valid.drop(['date'], axis=1)


X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train))
X_valid_scaled = pd.DataFrame(scaler.transform(X_valid))
list_X = X_train.columns.values.tolist()
X_train_scaled.set_axis(list_X, axis = 'columns', inplace=True)
X_valid_scaled.set_axis(list_X, axis = 'columns', inplace=True)


print(X_train_scaled)

       SO2op1    SO2op2         T       NO2
0    0.859881  0.031743  1.511389 -0.746816
1   -0.695262 -0.663402 -0.929485  0.466760
2    0.619063  0.874095  1.999564 -0.746816
3    0.472447  0.162594 -0.199175 -0.746816
4   -0.692746 -0.396248 -0.929485  0.466760
..        ...       ...       ...       ...
99  -0.698934 -0.717923 -0.683445  0.466760
100 -1.531742 -1.884676 -0.441310  1.680336
101  0.868926  1.124892  1.511389 -0.746816
102  0.469336 -0.161807 -0.199175 -0.746816
103 -0.693069 -0.529825 -0.929485  0.466760

[104 rows x 4 columns]


In [4]:
from keras import models
from keras import layers
from sklearn.datasets import make_regression
from sklearn import preprocessing
from sklearn.model_selection import RepeatedKFold
kfold = RepeatedKFold(n_splits=5, n_repeats=100)

# Type_error = 'mape'

List_Error = ['mean_absolute_percentage_error', 'mean_squared_error']
Type_error = 'mean_squared_error'

# ['softmax', 'softplus', 'softsign']
# Type_func = List_Function[0]
# Type_optimizer='adam'   
Type_optimizer='RMSprop'
Number_epochs = 100



from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import Dropout

l_feat = X_train_scaled.shape[1]
loss_func = 'mean_absolute_error'


print(X_train_scaled)


def network_model(l_r = 0.01, d_c = 0.001, reg = -1, activ = 'tanh',\
                  n_layer = 1, n_neur = 3, drop = 0):
    network = models.Sequential()
    for j in range(n_layer):
        network.add(layers.Dense(units = n_neur, activation = activ, input_shape=(l_feat,),\
                                 kernel_regularizer=regularizers.L1(10**reg)))
        network.add(Dropout(drop))
    network.add(layers.Dense(units = 1, activation = 'linear'))
    network.compile(loss = loss_func, optimizer = \
                    keras.optimizers.Adam(learning_rate = l_r, decay = d_c),\
                    metrics='mean_absolute_percentage_error') 
    return network

       SO2op1    SO2op2         T       NO2
0    0.859881  0.031743  1.511389 -0.746816
1   -0.695262 -0.663402 -0.929485  0.466760
2    0.619063  0.874095  1.999564 -0.746816
3    0.472447  0.162594 -0.199175 -0.746816
4   -0.692746 -0.396248 -0.929485  0.466760
..        ...       ...       ...       ...
99  -0.698934 -0.717923 -0.683445  0.466760
100 -1.531742 -1.884676 -0.441310  1.680336
101  0.868926  1.124892  1.511389 -0.746816
102  0.469336 -0.161807 -0.199175 -0.746816
103 -0.693069 -0.529825 -0.929485  0.466760

[104 rows x 4 columns]


In [ ]:
FFNN = KerasRegressor(build_fn = network_model, verbose=1)



parameters = {
                'l_r': [0.001,0.003,0.01, 0.03,  0.1],
                'd_c': [0.001,0.003, 0.01,0.03],
                'reg': [-1, -1.5,-2],
                'n_layer': [1,2,3,4],
                'n_neur': [2,3,4,5],
                'activ': ['tanh','relu'],
                'drop': [0, 0.2, 0.4, 0.6, 0.8],
                'batch_size': [4,8,16,32],
              # 'activ': ['linear', 'tanh', 'sigmoid', 'relu', 'exponential']
                'epochs': [5,10,20,40,80]
                }
#                            scoring='neg_mean_squared_error',
#                            scoring = "neg_mean_absolute_percentage_error",
grid =  GridSearchCV(estimator = FFNN, param_grid = parameters,\
                     scoring = "neg_mean_absolute_error", cv = kfold,\
                     n_jobs = -1)




grid_result = grid.fit(X_train_scaled, y_train)


print(grid_result.best_params_)
print(grid_result.best_score_)


# print(grid_result.cv_results_)

df_result = pd.DataFrame(grid_result.cv_results_)
print(df_result)

df_result.to_csv("GSCV/GSCV_1.csv", index=True)




2022-11-14 12:56:53.095581: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 12:56:53.095580: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 12:56:53.095580: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

In [ ]:
class LinearDataset(Dataset):
    def __init__(self, data_samples, data_targets):
        self.samples = data_samples
        self.targets = data_targets
        
    def __len__(self):
        return len(self.targets)


def load_data(data_samples, data_targets, t_s = 0.33):
    train_samples, valid_samples, train_targets, valid_targets =\
    train_test_split(data_samples, date_targets, test_size = t_s)
        
    train_data = LinearDataset(train_samples, train_targets)
    valid_data = LinearDataset(valid_samples, valid_targets)

    return train_data, test_data



In [ ]:
# def get_data(t_s = 0.33):
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = t_s)
#     time_train = X_train.dropna()['date']
#     X_train = X_train.drop(['date'], axis=1)
#     time_valid = X_valid.dropna()['date']
#     X_valid = X_valid.drop(['date'], axis=1)
#     X_train['SO2'] = y_train
#     X_valid['SO2'] = y_valid
#     return X_train, X_valid


# l_feat = X_train_scaled.shape[1]
# print(l_feat)

In [ ]:
def create_model(trial):
    num_layers = trial.suggest_int("num_layers", 1, 3)
    drop_out = trial.suggest_float("drop_out", 0, 1.0)
    model = tf.keras.Sequential()

    for i in range(n_layers):
        num_hidden = trial.suggest_int("n_units_l{}".format(i), 2, 5, log=False)
        model.add(tf.keras.layers.Dense(num_hidden, activation="relu"))
        network.add(Dropout(drop_out))
    
    network.add(layers.Dense(units = 1, activation = 'linear'))
#     network.compile(loss = 'mean_absolute_error', optimizer = \
#                     keras.optimizers.Adam(learning_rate = l_r, decay = d_c,\
#                                           beta_1 = b_1, beta_2 = b_2, epsilon = eps),\
#                     metrics='mean_absolute_percentage_error') 
    return model

In [ ]:
def create_optimizer(trial):
    kwargs = {}
    optimizer_options = ["Adam", "SGD"]
    optimizer_selected = trial.suggest_categorical("optimizer", optimizer_options)
    if optimizer_selected == "Adam":
        kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-5, 1e-1, log=True)
        kwargs["decay"] = trial.suggest_float("adam_decay", 1e-4, 1e-1, log = True)
    elif optimizer_selected == "SGD":
        kwargs["learning_rate"] = trial.suggest_float("SGD_learning_rate", 1e-5, 1e-1, log=True)
        kwargs["momentum"] = trial.suggest_float('SGD_momentum',1e-3,1e-1,log = True)
        kwargs["decay"] = trial.suggest_float("adam_decay", 1e-4, 1e-1, log = True)
    optimizer = getattr(tf.optimizers, optimizer_selected)(**kwargs)
    return optimizer

In [ ]:
def learn(model, optimizer, dataset, mode="eval"):
    
    mae = tf.keras.metrics.MeanAbsoluteError('mean_absolute_error', dtype=tf.float32)
#     accuracy = tf.metrics.Accuracy("accuracy", dtype=tf.float32)

    for batch, (sampleas, targets) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(sampleas, training=(mode == "train"))
            loss_value = ...
            ...

    if mode == "eval":
        return mae

In [ ]:
def objective(trial):
    # Get train/valid data.
    train_ds, valid_ds = load_data(X,y)

    # Build model and optimizer.
    model = create_model(trial)
    optimizer = create_optimizer(trial)

    # Training and validating cycle.
    for _ in range(epochs):
        learn(model, optimizer, train_ds, "train")

    accuracy = learn(model, optimizer, valid_ds, "eval")

    # Return last validation accuracy.
    return accuracy.result()

In [ ]:
def search():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader 
import torch.nn as nn
import torch.optim as optim

import numpy as np

from ray import tune
from ray.tune.schedulers import ASHAScheduler

class LinearDataset(Dataset):
    def __init__(self, data_samples, data_targets, device='cuda:0'):
        self.samples = data_samples
        self.targets = data_targets
        self.device=device
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        target = self.targets[idx]
        
        return torch.tensor(sample, device=self.device), torch.tensor(target, device=self.device)


def load_data(data_samples, data_targets):
    
    data = LinearDataset(data_samples, data_targets)

    # the constants to define
    test_rate = .2

    # the constants to calculate
    test_num = round(len(data)*test_rate)
    train_num = len(data) - test_num

    train_data, test_data = random_split(data, [train_num, test_num], generator=torch.Generator().manual_seed(42))   
    return train_data, test_data












In [ ]:
def build_model_custom(trial):
    
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 4
    
    for i in range(n_layers):
        
        out_features = trial.suggest_int("n_units_l{}".format(i), 1, 5)
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())

        in_features = out_features
        
    layers.append(nn.Linear(in_features, 1))
    layers.append(nn.ReLU())
    
    return nn.Sequential(*layers)

In [ ]:
def train_and_evaluate(param, model, trial):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.L1Loss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(EPOCHS):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)
            
            # Add prune mechanism
            trial.report(accuracy, epoch_num)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return accuracy



In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim


# Build a model by implementing define-by-run design from Optuna
def build_model_custom(trial):
    
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 4
    
    for i in range(n_layers):
        
        out_features = trial.suggest_int("n_units_l{}".format(i), 1, 5)
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())

        in_features = out_features
        
    layers.append(nn.Linear(in_features, 1))
    layers.append(nn.ReLU())
    
    return nn.Sequential(*layers)

# Train and evaluate the accuracy of neural network with the addition of pruning mechanism
def train_and_evaluate(param, model, trial):
    
#     df = pd.read_csv('heart.csv')
#     df = pd.get_dummies(df)
    
#     train_data, val_data = train_test_split(df, test_size = 0.2, random_state = 42)
#     train, val = Dataset(train_data), Dataset(val_data)

#     train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
#     val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

#     criterion = nn.CrossEntropyLoss()
    criterion = nn.L1Loss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(EPOCHS):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)
            
            # Add prune mechanism
            trial.report(accuracy, epoch_num)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return accuracy
  
# Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy
def objective(trial):

     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }
    
     model = build_model_custom(trial)

     accuracy = train_and_evaluate(params, model, trial)

     return accuracy
  
  
EPOCHS = 30
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=30)



In [ ]:
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from sklearn.datasets import make_regression

from sklearn import preprocessing

from sklearn.model_selection import RepeatedKFold
kfold = RepeatedKFold(n_splits=5, n_repeats=100)
from ann_visualizer.visualize import ann_viz

# Type_error = 'mape'

List_Error = ['mean_absolute_percentage_error', 'mean_squared_error']
Type_error = 'mean_squared_error'

# ['softmax', 'softplus', 'softsign']
# Type_func = List_Function[0]
# Type_optimizer='adam'   
Type_optimizer='RMSprop'
Number_epochs = 100



from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import Dropout

l_feat = X_train_scaled_poly.shape[1]
activ_func = 'tanh'
loss_func = 'mean_absolute_error'


print(X_train_scaled_poly)




b_1 = 0.9
b_2 = 0.999
eps = 1e-07
lr = 0.005
d = 0.005


def network_model(l_r = 0.01, d_c = 0.001, b_1 = 0.9, b_2 = 0.999, eps = 1e-01, reg = -1, activ = 'tanh',n_layer = 1, n_neur = 10, drop = 0):
    network = models.Sequential()
    for j in range(n_layer):
        network.add(layers.Dense(units = n_neur, activation = activ, input_shape=(l_feat,), kernel_regularizer=regularizers.L1(10**reg)))
        network.add(Dropout(drop))
    
    network.add(layers.Dense(units = 1, activation = 'linear'))
    network.compile(loss = 'mean_absolute_error', optimizer= keras.optimizers.Adam(learning_rate = l_r, decay = d_c, beta_1 = b_1, beta_2 = b_2, epsilon = eps), metrics='mean_absolute_percentage_error') 
    return network




FFNN = KerasRegressor(build_fn = network_model, verbose=0, batch_size = 50, epochs = 1000)



parameters = {
              # 'l_r': [0.01, 0.05,  0.1],
              # 'd_c': [0.001,0.005, 0.01],
              # 'b_1': [0.8, 0.9, 0.99],
              # 'b_2': [0.9, 0.99, 0.999],

              # 'reg': [-1, -1.5],
              # 'eps': [1e-07, 1e-01],
              'n_layer': [1,2,3,4],
              'n_neur': [5,10,15],
              'activ': ['tanh','relu'],
              'drop': [0, 0.2, 0.4, 0.6, 0.8],
            
              # 'activ': ['linear', 'tanh', 'sigmoid', 'relu', 'exponential']
              # 'epochs': [30, 100, 300, 1000],
                }

grid =  GridSearchCV(estimator = FFNN,
                           param_grid = parameters,
                           # scoring='neg_mean_squared_error',
                           # scoring = "neg_mean_absolute_error",
                           scoring = "neg_mean_absolute_percentage_error",
                           cv = 10, 
                           n_jobs = -1)


grid_result = grid.fit(X_train_scaled_poly, y_train)


print(grid_result.best_params_)
print(grid_result.best_score_)


# print(grid_result.cv_results_)

df_result = pd.DataFrame(grid_result.cv_results_)
print(df_result)

df_result.to_csv("GSCV/GSCV_1000_V7_ep.csv", index=True)
